In [3]:
import os

bundle_path = os.path.abspath("bundle-with-intermediate.pem")
os.environ["NODE_EXTRA_CA_CERTS"] = bundle_path

print("NODE_EXTRA_CA_CERTS =", os.environ["NODE_EXTRA_CA_CERTS"])

NODE_EXTRA_CA_CERTS = /home/lucas/Projects/car_dealership/bundle-with-intermediate.pem


In [6]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from playwright.async_api import async_playwright
import asyncio
import random

BASE = "https://www.freitasleiloeiro.com.br"


def parse_items(html: str):
    soup = BeautifulSoup(html, "lxml")
    items = []

    for card in soup.select("div.cardlote"):
        lote_el = card.select_one(".cardLote-lote")
        title_el = card.select_one(".cardLote-descVeic span")
        value_el = card.select_one(".cardLote-vlr")
        lance_el = card.select_one(".cardLote-lance")
        details_el = card.select_one(".cardLote-details span")
        link_el = card.select_one('a[href*="LoteDetalhes"]')

        items.append({
            "lote": lote_el.get_text(strip=True) if lote_el else None,
            "title": title_el.get_text(" ", strip=True) if title_el else None,
            "value": value_el.get_text(" ", strip=True) if value_el else None,
            "lance": lance_el.get_text(" ", strip=True) if lance_el else None,
            "details": details_el.get_text(" ", strip=True) if details_el else None,
            "details_url": urljoin(BASE, link_el["href"]) if link_el else None,
        })

    return items


async def fetch_all_lots(leilao_id=7659, top_rows=12, max_pages=50):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            locale="pt-BR",
            user_agent=(
                "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/121.0 Safari/537.36"
            ),
        )
        page = await context.new_page()

        # Step 1: entry page (passes gocache/WAF)
        await page.goto(
            f"{BASE}/Leiloes/Lotes?leilaoId={leilao_id}",
            wait_until="domcontentloaded",
        )
        await page.wait_for_timeout(1500)

        all_items = []

        # Step 2: paginate ListarLotes
        for page_number in range(1, max_pages + 1):
            url = (
                f"{BASE}/Leiloes/ListarLotes"
                f"?LeilaoId={leilao_id}"
                f"&Nome=&Lote=&Categoria=1&TipoLoteId=1&FaixaValor=0"
                f"&Condicao=&PatioId=0&AnoModeloMin=2018&AnoModeloMax=0"
                f"&ArCondicionado=false&DirecaoAssistida=false"
                f"&Tag=&ClienteSclId=0"
                f"&PageNumber={page_number}&TopRows={top_rows}"
            )
            if page_number > 1:
                await asyncio.sleep(random.uniform(0.5, 1.5))

            resp = await page.request.get(
                url,
                headers={
                    "X-Requested-With": "XMLHttpRequest",
                    "Referer": f"{BASE}/Leiloes/Lotes?leilaoId={leilao_id}",
                    "Accept": "text/html,*/*;q=0.8",
                },
            )

            if resp.status != 200:
                print(f"Stopped at page {page_number} (status {resp.status})")
                break

            html = await resp.text()
            items = parse_items(html)

            if not items:
                break

            all_items.extend(items)

            # polite pacing
            await page.wait_for_timeout(600)

        await browser.close()
        return all_items

In [7]:
items = await fetch_all_lots(leilao_id=7659, max_pages=1)
len(items)

12

In [8]:
items

[{'lote': '002',
  'title': 'CHEV/ONIX 10MT LT2, 20/21, PLACA: R__-___3, GASOL/ALC, BRANCA',
  'value': 'R$ 32.500,00',
  'lance': 'Maior lance',
  'details': 'FINANCEIRA',
  'details_url': 'https://www.freitasleiloeiro.com.br/Leiloes/LoteDetalhes?leilaoId=7659&loteNumero=2'},
 {'lote': '004',
  'title': 'RENAULT/KWID ZEN 2, 22/23, PLACA: R__-___8, GASOL/ALC, BRANCA',
  'value': 'R$ 26.000,00',
  'lance': 'Maior lance',
  'details': 'FINANCEIRA',
  'details_url': 'https://www.freitasleiloeiro.com.br/Leiloes/LoteDetalhes?leilaoId=7659&loteNumero=4'},
 {'lote': '005',
  'title': 'FIAT/ARGO TREKKING 1.3AT, 23/24, PLACA: S__-___9, GASOL/ALC, PRETA',
  'value': 'R$ 47.000,00',
  'lance': 'Lance Inicial',
  'details': 'FINANCEIRA',
  'details_url': 'https://www.freitasleiloeiro.com.br/Leiloes/LoteDetalhes?leilaoId=7659&loteNumero=5'},
 {'lote': '006',
  'title': 'FIAT/MOBI LIKE, 19/20, PLACA: Q__-___4, GASOL/ALC, BRANCA',
  'value': 'R$ 23.000,00',
  'lance': 'Maior lance',
  'details': 'FIN

In [20]:
import pandas as pd
import re
def parse_title(title: str) -> dict:
    s = str(title)

    # brand / model
    m = re.match(r"^\s*([^/]+)/([^,]+)", s)
    brand = m.group(1).strip() if m else None
    model_raw = m.group(2).strip() if m else None

    # year model from "23/24" (take last 2 digits after slash)
    y = re.search(r"\b(\d{2})/(\d{2})\b", s)
    if y:
        yy = int(y.group(2))
        year_model = (1900 + yy) if yy > 70 else (2000 + yy)
    else:
        year_model = None

    # plate (keep as string)
    p = re.search(r"PLACA:\s*([^,]+)", s, flags=re.I)
    plate = p.group(1).strip() if p else None

    # fuel and color (by position after plate; safer: capture last two comma-separated parts)
    parts = [x.strip() for x in s.split(",")]
    fuel_raw = parts[-2] if len(parts) >= 2 else None
    color = parts[-1] if len(parts) >= 1 else None

    return {
        "Marca_raw": brand,
        "Modelo_raw": model_raw,
        "AnoModelo": year_model,
        "Placa": plate,
        "Combustivel_raw": fuel_raw,
        "Cor": color,
    }


In [24]:
df_ = pd.DataFrame(items)
df_parsed = df_["title"].apply(parse_title).apply(pd.Series)
df_final = pd.concat([df_, df_parsed], axis=1)


In [27]:
df_final[:1]

,lote,title,value,lance,details,details_url,Marca_raw,Modelo_raw,AnoModelo,Placa,Combustivel_raw,Cor
0,002,"CHEV/ONIX 10MT LT2, 20/21, PLACA: R__-___3, GA...","R$ 32.500,00",Maior lance,FINANCEIRA,https://www.freitasleiloeiro.com.br/Leiloes/Lo...,CHEV,ONIX 10MT LT2,2021,R__-___3,GASOL/ALC,BRANCA


In [10]:
import pandas as pd

df = pd.read_csv("fipe_data_330.csv")

In [23]:
df[df['Marca'].str.contains("Renault") & df['AnoLabel'].str.contains("2023") & df['Modelo'].str.contains("KWID") ]

,Modelo,CodigoModelo,AnoLabel,AnoModelo,Combustivel,Valor,CodigoFipe,MesReferencia,DataConsulta,Valor_num,Marca
1393,KWID Intense (Elétrico),10022,2023 Elétrico,2023.0,Elétrico,"R$ 73.442,00",025318-9,fevereiro de 2026,"domingo, 15 de fevereiro de 2026 06:31",73442.0,Renault
1397,KWID Intense 1.0 Flex 12V 5p Mec.,8021,2023 Flex,2023.0,Flex,"R$ 52.028,00",025267-0,fevereiro de 2026,"domingo, 15 de fevereiro de 2026 06:32",52028.0,Renault
1411,KWID OUTSIDER 1.0 Flex 12V 5p Mec.,8701,2023 Flex,2023.0,Flex,"R$ 56.060,00",025279-4,fevereiro de 2026,"domingo, 15 de fevereiro de 2026 06:35",56060.0,Renault
1418,KWID Zen 1.0 Flex 12V 5p Mec.,8023,2023 Flex,2023.0,Flex,"R$ 49.888,00",025266-2,fevereiro de 2026,"domingo, 15 de fevereiro de 2026 06:36",49888.0,Renault


In [ ]:
import pandas as pd
from rapidfuzz import fuzz, process


def match_simple(
    df_auc: pd.DataFrame,
    df_fipe: pd.DataFrame,
    score_cutoff: int = 20,
    top_n: int = 5,
) -> pd.DataFrame:

    # shallow copies
    a = df_auc.copy()
    f = df_fipe.copy()

    a["ano_key"] = pd.to_numeric(a["AnoModelo"], errors="coerce").astype("Int64")
    f["ano_key"] = pd.to_numeric(f["AnoModelo"], errors="coerce").astype("Int64")

    # pre-index FIPE by year for speed
    f_by_year = {y: g for y, g in f.groupby("ano_key", dropna=True)}

    rows = []
    for i, r in a.iterrows():
        year = r["ano_key"]
        if pd.isna(year) or year not in f_by_year:
            continue

        # 1) filter candidates by same year
        cand = f_by_year[year]
        
        # 2) brand contains (FIPE Marca contains marca_raw)
        # e.g., "VW - VOLKSWAGEN" contains "VW"
        marca = str(r["Marca_raw"]).lower()
        cand = cand[cand["Marca"].str.lower().str.contains(marca, na=False)]

        if cand.empty:
            continue

        # 3) fuzzy match model
        # use token_set_ratio for robustness to word order / extra tokens
        choices = cand["Modelo"].tolist()
        results = process.extract(
            query=r["Modelo_raw"],
            choices=choices,
            scorer=fuzz.token_set_ratio,
            limit=top_n,
            score_cutoff=score_cutoff,
        )


        # 4) collect matches (link back to FIPE rows)
        # results items are (matched_string, score, index_in_choices)
        for matched_model_key, score, pos in results:
            fipe_row = cand.iloc[pos]

            value_str = r.get("value")
            value_num = pd.to_numeric(
                str(value_str).replace("R$", "").replace(".", "").replace(",", ".").strip(),
                errors="coerce",
            )

            rows.append({
                "auc_index": i,
                "Marca_raw": r["Marca_raw"],
                "Modelo_raw": r["Modelo_raw"],
                "value": value_num,
                "AnoModelo": int(year),

                "score": float(score),

                # FIPE fields you care about:
                "fipe_Marca": fipe_row["Marca"],
                "fipe_Modelo": fipe_row["Modelo"],
                "fipe_AnoModelo": int(fipe_row["AnoModelo"]) if pd.notna(fipe_row["AnoModelo"]) else None,
                "fipe_CodigoFipe": fipe_row.get("CodigoFipe"),
                "fipe_CodigoModelo": fipe_row.get("CodigoModelo"),
                "fipe_Valor_num": fipe_row.get("Valor_num"),
            })

    out = pd.DataFrame(rows)
    if not out.empty:
        out = out.sort_values(["auc_index", "score"], ascending=[True, False])
    return out

In [56]:

df_out = match_simple(df_final[:1], df)

Matches for AUC index 0 (Marca='CHEV', Modelo='ONIX 10MT LT2', AnoModelo=2021):


In [57]:
df_out

,auc_index,Marca_raw,Modelo_raw,AnoModelo,score,fipe_Marca,fipe_Modelo,fipe_AnoModelo,fipe_CodigoFipe,fipe_CodigoModelo,fipe_Valor_num
0,0,CHEV,ONIX 10MT LT2,2021,47.058824,GM - Chevrolet,ONIX HATCH 1.0 12V Flex 5p Mec.,2021,004519-5,8988,60432.0
1,0,CHEV,ONIX 10MT LT2,2021,47.058824,GM - Chevrolet,ONIX HATCH 1.0 12V TB Flex 5p Aut.,2021,004511-0,8889,67003.0
2,0,CHEV,ONIX 10MT LT2,2021,47.058824,GM - Chevrolet,ONIX HATCH LT 1.0 12V Flex 5p Mec.,2021,004517-9,8949,59692.0
3,0,CHEV,ONIX 10MT LT2,2021,47.058824,GM - Chevrolet,ONIX HATCH LT 1.0 12V TB Flex 5p Aut.,2021,004514-4,8890,69097.0
4,0,CHEV,ONIX 10MT LT2,2021,47.058824,GM - Chevrolet,ONIX HATCH LT 1.0 12V TB Flex 5p Mec.,2021,004512-8,8891,67262.0
